Author: Alex Cohen Dambrós Lopes 

=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Getting all light curves from space telescopes and running their pre-processing and data analysis
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

This is an example code used to obtain light curves from space telescopes and preprocess 
them automatically


# Imports

In [ ]:
import pandas as pd
import numpy as np
import warnings
import lightkurve as lk
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
from Functions import all_functions

# General Functions

In [ ]:
def add_value_labels(ax, spacing=5):
    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.0f}".format(y_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va) 

# Presets

In [ ]:
# ============= RandomState =============
random_state = np.random.RandomState(123)


# ============= Warnings =============
warnings.simplefilter("ignore")


pd.set_option('display.max_columns', None)  # show all cols
pd.set_option('display.max_rows', None) # show all rows
pd.set_option("expand_frame_repr", False) # print cols side by side as it's supposed to be

# Download Data

In [ ]:
# all_functions.download_all_datasets()

# Open Datasets

In [ ]:
df_tess = all_functions.read_dataset('tess')
df_kepler = all_functions.read_dataset('kepler')
df_k2 = all_functions.read_dataset('k2')

# Initial Analysis

## Head of Data

In [ ]:
df_tess.head()

In [ ]:
df_kepler.head()

In [ ]:
df_k2.head()

In [ ]:
#  Rename specific columns
df_tess.rename(columns={"tfopwg_disp": "disposition"}, inplace = True)
df_kepler.rename(columns={"koi_disposition": "disposition"}, inplace = True)

## Data Visualization

In [ ]:
telescope_data_list = {"TESS": df_tess, "KEPLER": df_kepler, "K2": df_k2}

In [ ]:
"""
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Total number of instances and columns
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
"""

list_disposition = ['FALSE POSITIVE', 'CONFIRMED']

for telescope, df_telescope in telescope_data_list.items():
    print(f"============ {telescope} ============")
    print(f"For the data collected by the {telescope} space telescope, it has {df_telescope.shape[1]} columns and {df_telescope.shape[0]} instances.")
    
    """
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    Percentage of false positives and candidates
    =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
    """
    
    df_filtered = df_telescope[(df_telescope['disposition'] == 'FALSE POSITIVE') | (df_telescope['disposition'] == 'CONFIRMED')]
    sum_false_positives = (df_filtered['disposition'] == 'FALSE POSITIVE').sum()
    sum_confirmed = (df_filtered['disposition'] == 'CONFIRMED').sum()
    print("Total False Positives: %d, Total Confirmed: %d, Others: %d" % (sum_false_positives, sum_confirmed, df_telescope.shape[0] - (sum_confirmed + sum_false_positives)))
    print("Percentage False Positives: %.2f%%, Percentage Confirmed: %.2f%%, Percentage of Others: %.2f%%" % ((sum_false_positives / df_telescope.shape[0]) * 100, (sum_confirmed / df_telescope.shape[0]) * 100, (df_telescope.shape[0] - (sum_confirmed + sum_false_positives)) * 100 / df_telescope.shape[0]))
    

In [ ]:
# TESS
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(df_tess["disposition"], color = "#0092D4", ax=ax)
add_value_labels(ax)
plt.title("Amount of disposition of the data present in the TESS telescope", loc="left")
plt.xlabel("Disposition")
plt.ylabel("Amount")
plt.ylim(0, 5000)
plt.show()

In [ ]:
# KEPLER
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(df_kepler["disposition"], color = "#0092D4", ax=ax)
add_value_labels(ax)
plt.title("Amount of disposition of the data present in the KEPLER telescope", loc="left")
plt.xlabel("Disposition")
plt.ylabel("Amount")
plt.ylim(0, 5200)
plt.show()

In [ ]:
# K2
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(df_k2["disposition"], color = "#0092D4", ax=ax)
add_value_labels(ax)
plt.title("Amount of disposition of the data present in the K2 telescope", loc="left")
plt.xlabel("Disposition")
plt.ylabel("Amount")
plt.ylim(0, 2500)
plt.show()

## Percentage of null values ​​in each column of the TESS telescope

In [ ]:
null_values = round((df_tess.isna().sum() / len(df_tess)) * 100, 2)
null_values = null_values.to_frame()
null_values.rename(columns={0: "%null"}, inplace = True)
null_values

# Percentage of null values ​​in each column of the KEPLER telescope

In [ ]:
null_values = round((df_kepler.isna().sum() / len(df_kepler)) * 100, 2)
null_values = null_values.to_frame()
null_values.rename(columns={0: "%null"}, inplace = True)
null_values

# Percentage of null values ​​in each column of the K2 telescope

In [ ]:
null_values = round((df_k2.isna().sum() / len(df_k2)) * 100, 2)
null_values = null_values.to_frame()
null_values.rename(columns={0: "%null"}, inplace = True)
null_values

# Selecting specific columns for the first test

In [ ]:
# drop rows of planets that were discovered by methods other than transit
df_k2 = df_k2[df_k2['discoverymethod'] != 'Radial Velocity']

In [ ]:
# TESS
df_tess  = df_tess[['tid', 'disposition', 'pl_orbper', 'pl_trandurh']]
df_tess.rename(columns={"tid": "id_target", "pl_orbper": "period", "pl_trandurh": "duration"}, inplace = True)

# KEPLER
df_kepler = df_kepler[['kepid', 'disposition','koi_period','koi_duration', 'koi_time0bk']]
df_kepler.rename(columns={"kepid": "id_target", "koi_period": "period", "koi_duration": "duration"}, inplace = True)

# K2
df_k2 = df_k2[['tic_id', 'disposition', 'pl_orbper', 'pl_trandur']]
df_k2.rename(columns={"tic_id": "id_target", "pl_orbper": "period", "pl_trandur": "duration"}, inplace = True)

In [ ]:
df_tess.head()

# Local View & Global View

- global view -> All light curves are binned to the same length

- local view -> so that the transit occupies a fixed fraction of the resulting vector.

The global view has a fixed length of 2001 bins, while the local view has a fixed length of 201 bins.

In [ ]:
telescopes_list = {'Kepler': df_kepler, 'TESS': df_tess, 'K2': df_k2}

for name_telescope, df_telescope in telescopes_list.items():
    for index, row in df_telescope.iterrows():
        try:
            id_target, period, duration, t0 = row[0], row[2], row[3], row[4]
        except:
            id_target, period, duration = row[0], row[2], row[3]
            
        try:
            
            if name_telescope == 'Kepler':
                id_target = 'KIC ' + str(id_target)
            elif name_telescope == 'TESS':
                id_target = 'TIC ' + str(id_target)
            
                
            lcs = lk.search_lightcurve(id_target, mission= name_telescope, cadence='long').download_all()
            
            if not(lcs is None):
                
                # This method concatenates all quarters in our LightCurveCollection together, and normalizes them at the same time.
                lc_raw = lcs.stitch()
                
                # Clean outliers, but only those that are above the mean level (e.g. attributable to stellar flares or cosmic rays).
                lc_clean = lc_raw.remove_outliers(sigma=20, sigma_upper=4)
                
                
                # We have to mask the transit to avoid self-subtraction the genuine planet signal when we flatten the lightcurve. We have to do a hack to find where the time series should be masked.
                if t0 != None:
                    temp_fold = lc_clean.fold(period, epoch_time=t0)
                else:
                    temp_fold = lc_clean.fold(period)
            
                fractional_duration = (duration / 24.0) / period
                phase_mask = np.abs(temp_fold.phase.value) < (fractional_duration * 1.5)
                transit_mask = np.in1d(lc_clean.time.value, temp_fold.time_original.value[phase_mask])
            
                # Now flatten the mask, but interpolate the trend over the points possessing the transit signal of interest.
                lc_flat, trend_lc = lc_clean.flatten(return_trend=True, mask=transit_mask)
                
                # Now fold the cleaned, flattened lightcurve:
                if t0 != None:
                    lc_fold = lc_flat.fold(period, epoch_time=t0)
                else:
                    lc_fold = lc_flat.fold(period)
                    
                
                # ========= Defining global curves =========
                lc_global = lc_fold.bin(time_bin_size=0.005).normalize() - 1
                lc_global = (lc_global / np.abs(lc_global.flux.min()) ) * 2.0 + 1
                
                phase_mask = (lc_fold.phase > -4*fractional_duration) & (lc_fold.phase < 4.0*fractional_duration)
                lc_zoom = lc_fold[phase_mask]
                
                # ========= Defining local curves =========
                lc_local = lc_zoom.bin(bins=201).normalize() - 1
                lc_local = (lc_local / np.abs(np.nanmin(lc_local.flux)) ) * 2.0 + 1
                
                
                print(f"{id_target} target pre-processing performed")
                break
            
            else:
                print("Error downloading target data:", id_target)
        
        except Exception as error:
            print(error)